## The goal is to fetch Bicol Region sea route transport network

### 0. Install dependencies

In [67]:
from folium.plugins import MarkerCluster
import pandas as pd
import osmnx as ox
import networkx as nx
import folium
import os

### 1. Define the places and create a directed empty multigraph to combine all places

In [72]:
# Define the individual places in Bicol Region
places = ["Catanduanes, Philippines", "Masbate, Philippines", "Albay, Philippines"]

# Create an empty directed multigraph to combine all places
combined_graph = nx.MultiDiGraph()

# Loop through each place and retrieve the graph for ferry routes
for place in places:
    # Use a filter specifically for ferry routes
    graph = ox.graph_from_place(place, custom_filter='["route"~"ferry"]', network_type='all')
    combined_graph = nx.compose(combined_graph, graph)

# Get the centroid of one of the places to initialize the map
centroid = ox.geocode_to_gdf(places[0]).unary_union.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=8, tiles='OpenStreetMap')

# Get the edges GeoDataFrame (for ferry routes)
gdf_edges = ox.graph_to_gdfs(combined_graph, nodes=False, edges=True)

# Add the edges (ferry routes) to the map
for _, edge in gdf_edges.iterrows():
    points = [(y, x) for x, y in edge['geometry'].coords]
    folium.PolyLine(points, color="blue", weight=2.5, opacity=0.8).add_to(m)

# Create the directory if it doesn't exist
if not os.path.exists("html-outputs"):
    os.makedirs("html-outputs")

# Save the map inside the 'html-outputs' folder
m.save("html-outputs/bicol_ferry_routes.html")

C:\Users\ADMIN\miniconda3\envs\ox\Lib\site-packages\osmnx\_overpass.py:254: UserWarning: This area is 17 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


### 2. Loop through each place and retrieve the graph for each